<a href="https://colab.research.google.com/github/futurefallprabhas/Fraudulent-Transaction-Analysis/blob/codespace-jubilant-waddle-g445w6grq4xph6jr/Assignment1_Set1_2_ML_14_Mar_RJ_v4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Import Libraries/Dataset

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

#data=pd.read_csv('ML_Assignment_1.csv')
#data = pd.read_csv(r'D:\mtech_bits_pilani\sem2\ml\assignment\Assignment 1 Set 1 Dataset.csv')
data = pd.read_csv('C:\\Users\\Rajkumar\\Desktop\\GoldenVisa\\DataScience-Bits-Pilani\\Mtech-Course-Start\\2-semester-start-real\\1-Machine Learning-DSECLZG565\\Assignments\\Assignment1\\Financial_Transaction.csv',index_col=[0])

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Rajkumar\\Desktop\\GoldenVisa\\DataScience-Bits-Pilani\\Mtech-Course-Start\\2-semester-start-real\\1-Machine Learning-DSECLZG565\\Assignments\\Assignment1\\Financial_Transaction.csv'

In [ ]:
data.head(10)

### 2. Data Visualization and Exploration

#### a. Print 2 rows for sanity check to identify all the features present in the dataset

In [ ]:
# a.Print the first 2 rows of the dataset
data.head(2)


In [ ]:
data.columns.tolist()

In [ ]:
class_counts = data['Is Fraudulent'].value_counts()
print(class_counts)

#### b. Comment on class imbalance with appropriate visualization method.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# Count the occurrences of each class
class_counts = data['Is Fraudulent'].value_counts()

# Plot the distribution of classes
plt.figure(figsize=(8, 6))
sns.barplot(x=class_counts.index, y=class_counts.values)
plt.title('Distribution of Classes')
plt.xlabel('Class')
plt.ylabel('Count')
plt.xticks([0, 1], ['Non-Fraudulent', 'Fraudulent'])
plt.show()


Class Imbalance:
As we can see from the above image , the non-fraudlent class makes up the majority class and the fraudulent makes up the minority class. Hence our data set is imbalanced, that can result to biased predictions.

In [ ]:
#Get instances of fraud and normal transactions
fraud = data[data['Is Fraudulent'] == 1]
no_fraud = data[data['Is Fraudulent'] == 0]
print("There are", len(fraud), "instances of fraud")
print("There are", len(no_fraud), "normal transactions")

In [ ]:
#Get summary statistics for the amount paid
fraud.Amount.describe()

In [ ]:
no_fraud.Amount.describe()

In [ ]:
#Compare both transaction values
data.groupby('Is Fraudulent').mean()

#### C. Provide appropriate data visualizations to get an insight about the dataset.

In [ ]:
# Histogram for numerical features
data.hist(figsize=(12, 10))
plt.tight_layout()
plt.show()


In [ ]:
# Count plot for categorical features
plt.figure(figsize=(10, 6))
sns.countplot(x='Card Type', data=data)
plt.title('Count of Transactions by Card Type')
plt.xlabel('Card Type')
plt.ylabel('Count')
plt.show()


In [ ]:
# Box plot of 'Amount' by 'Is Fraudulent'
plt.figure(figsize=(8, 6))
sns.boxplot(x='Is Fraudulent', y='Amount', data=data)
plt.title('Amount Distribution by Fraudulent Status')
plt.xlabel('Is Fraudulent')
plt.ylabel('Amount')
plt.xticks([0, 1], ['Non-Fraudulent', 'Fraudulent'])
plt.show()


In [ ]:
#Visualize pairwise relationships between numerical features and the target variable
sns.pairplot(data, hue='Is Fraudulent', vars=['Amount', 'Previous Transactions', 'Balance Before Transaction'])
plt.show()


#### d. Do the correlational analysis on the dataset. Provide a visualization for the same

In [ ]:
# Correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(data.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap')
plt.show()


Justification :
Correlation analysis helps us in providing the following insights while doing feature engineering:
    Features with high correlation are more linearly dependent and hence have almost the same effect on the dependent variable. So, when two features have high correlation, we can drop one of the two features. This analysis helps is identifying those redundant features.
    Helps idetify the featuers that are highly correlated with the target variable. Using these features have a significant impact on the outcome.
    Correlation analysis helps in detecting multicollinearity, which occurs when independent variables are highly correlated with each other. Multicollinearity can cause issues in regression analysis, making it difficult to determine the effect of each variable independently. In such cases, you might choose to remove one of the correlated variables during feature selection to alleviate multicollinearity.
    Correlation analysis can aid in dimensionality reduction by selecting a subset of features that are most relevant for prediction while discarding less informative or redundant features. This can lead to simpler and more interpretable models.

    Each cell in the heatmap represents the correlation coefficient between two variables. The values range from -1 to 1, where:
•	1 indicates a perfect positive correlation

•	-1 indicates a perfect negative correlation

•	0 indicates no correlation

Now, regarding the effect of correlational analysis on feature selection:
1.	Identifying Redundant Features: Highly correlated features may provide redundant information. In feature selection, redundant features can be eliminated to reduce model complexity and improve interpretability without sacrificing predictive performance.
2.	Avoiding Multicollinearity: Correlated features can lead to multicollinearity, which can destabilize model coefficients and make the model's estimates less reliable. Removing one of the highly correlated features can mitigate multicollinearity issues.
3.	Improving Model Performance: Removing irrelevant or redundant features based on correlational analysis can lead to better model performance by focusing on the most informative features. It helps in building simpler and more interpretable models, reducing the risk of overfitting and improving generalization to unseen data.

In summary, correlational analysis plays a crucial role in feature selection by helping to identify and remove redundant or irrelevant features, thereby improving model performance, interpretability, and generalization.


## 3. Data Pre-processing and cleaning

#### a. Do the appropriate pre-processing of the data like identifying NULL or Missing Values if any, handling of outliers if present in the dataset, skewed data etc. Mention the pre-processing steps performed in the markdown cell. Explore few latest data balancing tasks and its effect on model evaluation parameters.

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

Financial Transaction data set has no null values

In [ ]:
print("Financial Transaction data set has", data.isnull().values.sum(), "missing values")

In [ ]:
data.describe()

#### Checking outliers using boxplots

In [ ]:

# Identify categorical columns
categorical_columns = data.select_dtypes(include=['object']).columns.tolist()

# List of columns to exclude from the boxplot generation
columns_to_exclude = ['Is Fraudulent']


# Combine columns to exclude
columns_to_exclude.extend(categorical_columns)

num_cols_per_row = 3  # Define num_cols_per_row before the loop
plot_count = 0  # Initialize plot_count before the loop

for column in data.columns:
    if column not in columns_to_exclude:  # Exclude specified columns
        if data[column].nunique() > 1:  # Ensure the column has more than one unique value
            # Create a new row of plots if the number of plots in the current row exceeds num_cols_per_row
            if plot_count % num_cols_per_row == 0:
                fig, axes = plt.subplots(1, num_cols_per_row, figsize=(15, 5))  # Adjust the figure size as needed
            # Generate boxplot and add it to the current row
            data[[column]].boxplot(ax=axes[plot_count % num_cols_per_row])
            axes[plot_count % num_cols_per_row].set_title('Boxplot of ' + column)
            axes[plot_count % num_cols_per_row].set_ylabel('Value')
            plot_count += 1
            # Add marks for outliers
            outliers = data[column][data[column] > data[column].quantile(0.75) + 1.5 * (data[column].quantile(0.75) - data[column].quantile(0.25))]
            axes[plot_count % num_cols_per_row].scatter([1] * len(outliers), outliers, color='red', marker='o')  # Add red circles for outliers
            # Save the figure when the row is complete
            if plot_count % num_cols_per_row == 0:
                plt.tight_layout()
                #plt.savefig(f'boxplots_row_{plot_count // num_cols_per_row}.png')  # Save each row of boxplots as an image
                #plt.close()  # Close the current figure to release memory

# Save the remaining plots if they don't fill a complete row
if plot_count % num_cols_per_row != 0:
    plt.tight_layout()
    #plt.savefig(f'boxplots_row_{plot_count // num_cols_per_row + 1}.png')  # Save the last row of boxplots as an image
    #plt.close()  # Close the current figure to release memory

In [ ]:
outliers

no outliers found for the given data from the boxplots

In [ ]:
#### Code to check for outliers based on IQR

In [ ]:
# Identify categorical columns
categorical_columns = data.select_dtypes(include=['object']).columns.tolist()

# List of columns to exclude from the boxplot generation
columns_to_exclude = ['Is Fraudulent']

# Combine columns to exclude
columns_to_exclude.extend(categorical_columns)

# Function to detect outliers using the IQR method
def detect_outliers_iqr(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = (series < lower_bound) | (series > upper_bound)
    return outliers

# Detect outliers in all columns
outliers = data.drop(columns=columns_to_exclude).apply(detect_outliers_iqr)

# Flag to track if any outliers were found
outliers_found = False

# Print columns with outliers
for column in outliers.columns:
    if outliers[column].any():
        outliers_found = True
        print(f"Outliers detected in column '{column}':")
        print(data[outliers[column]])
        print("\n")

# Print message if no outliers were found
if not outliers_found:
    print("No outliers detected in any column.")



In [ ]:
# Identify categorical columns
categorical_columns = data.select_dtypes(include=['object']).columns.tolist()

# List of columns to exclude from the boxplot generation
columns_to_exclude = ['Is Fraudulent']

# Combine columns to exclude
columns_to_exclude.extend(categorical_columns)


# Calculate skewness for each specified column
skewness = data.drop(columns=columns_to_exclude).skew()

# Print column name and skewness for each specified column
for column, skew in skewness.items():
    print(f"Skewness for {column}: {skew}")

#### The skweness has been calculated for all columns and it is found that the values are near to 0 and hence values
#### of the columns are uniformily distributed


In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data['Spending Patterns'], kde=True, color='blue', bins=75)  # You can adjust the number of bins for better visualization
plt.title('Spending Patterns')
plt.xlabel('Spending Pattern')
plt.ylabel('Density')
plt.show()

In [ ]:
financial_data_1 = data["Spending Patterns"]
financial_data_2 = data["Previous Transactions"]

plt.figure(figsize=(8, 6))
plt.boxplot(financial_data_1)
plt.xlabel("Financial Data")
plt.ylabel("Value")
plt.title("Box Plot based on Spending patterns")
plt.grid(True)
plt.show()

In [ ]:
#subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

#fraud transactions
sns.histplot(data[data['Is Fraudulent'] == 1]['Time of Day'], bins=50, kde=True, color='red', label='Fraud', ax=ax1)
ax1.set_title('Transaction Time of day for Fraud Transactions')
ax1.set_xlabel('Time in Hours')
ax1.set_ylabel('Number of Transactions')
ax1.legend()

#normal transactions
sns.histplot(data[data['Is Fraudulent'] == 0]['Time of Day'], bins=50, kde=True, color='blue', label='Normal', ax=ax2)
ax2.set_title('Transaction Times for Normal Transactions')
ax2.set_xlabel('Time in Days')
ax2.set_ylabel('Number of Transactions')
ax2.legend()

plt.tight_layout()
plt.show()

b.Apply appropriate feature engineering techniques for them. Apply the feature transformation techniques like Standardization, Normalization, etc. You are free to apply the appropriate transformations depending upon the structure and the complexity of your dataset.

In [ ]:
data.dtypes

In [ ]:
data_cat = data.select_dtypes('object')
data_num = data.select_dtypes(np.number)

In [ ]:
data_num

In [ ]:
data_cat

In [ ]:
# Function for one hot encoding

def one_hot_encode_categorical_columns(df_encod):
    # Identify categorical columns
    categorical_columns = df_encod.select_dtypes(include=['object']).columns.tolist()
    categorical_columns = [col for col in categorical_columns if col not in ('Date')]

    # Perform one-hot encoding for each categorical column
    for column in categorical_columns:
        df_encod = pd.concat([df_encod, pd.get_dummies(df_encod[column], prefix=column, drop_first=True)], axis=1)
        df_encod = df_encod.drop(column, axis=1)

    return df_encod

In [ ]:
data.info()

In [ ]:
df_encoded=one_hot_encode_categorical_columns(data)

In [ ]:
df_encoded.head()

In [ ]:
df_encoded.shape

In [ ]:
# Remvoving rows with NaN values:
df_encoded=df_encoded.dropna()
df_encoded.shape

In [ ]:
df_encoded.dtypes

In [ ]:
legit_data = df_encoded[df_encoded['Is Fraudulent'] == 0]
fraud_data = df_encoded[df_encoded['Is Fraudulent'] == 1]

In [ ]:
print(legit_data.shape)
print(fraud_data.shape)

In [ ]:
legit_data.Amount.describe()

In [ ]:
fraud_data.Amount.describe()

In [ ]:
df_encoded.groupby('Is Fraudulent').mean()

In [ ]:
# X defined with features other then target variable:
X=df_encoded.drop(['Is Fraudulent','Date'],axis=1)


# Y defined with the target variable
Y=df_encoded['Is Fraudulent']

# Split features and target variable
#X = data_ml.drop(['Is Fraudulent','Date'], axis=1)
#y = data_ml['Is Fraudulent']

In [ ]:
# SPlit training and testing data
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y, random_state=2)


In [ ]:
print(X.shape, X_train.shape, X_test.shape)

In [ ]:
# Model Training
# Logistic Regression

model = LogisticRegression(max_iter=5000)

model.fit(X_train,Y_train)

In [ ]:
# Model Evaluation

#Accuracy Score
X_train_pred = model.predict(X_train)
traininig_data_accuracy = accuracy_score(X_train_pred,Y_train)

In [ ]:
print('Accuracy Training data:  ',traininig_data_accuracy)

In [ ]:
# Accuracy on test data

X_test_pred = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_pred, Y_test)

In [ ]:
print('Accuracy score of Test data:  ',test_data_accuracy)

In [ ]:
#Get Correlation of "Churn" with other variables annd recording the observations
plt.figure(figsize=(15,8))
df_encoded.corr()['Is Fraudulent'].sort_values(ascending = False).plot(kind='bar')

In [ ]:
sns.heatmap(data.corr(), cmap='viridis')

**4.Model Building**

a. Split the dataset into training and test sets.

In [ ]:
# Split features and target variable - Train = 80 % Test = 20%
X = df_encoded.drop(['Is Fraudulent','Date'], axis=1)
y = df_encoded['Is Fraudulent']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

#Apply standard scalar
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)


print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

In [4]:
print(f"Training dataset: X_train: {X_train.shape}, y_train: {y_train.shape}\n{'_'*58}")
print(f"Testing data set: X_test: {X_test.shape}, y_test: {y_test.shape}")

NameError: name 'X_train' is not defined

In [ ]:
# Get distributions - Calculate the percentage of each class in the training set
train_class_percentage = y_train.value_counts(normalize=True) * 100
print("Class distribution in Training Set:")
print(train_class_percentage)

test_class_percentage = y_test.value_counts(normalize=True) * 100
print("\nClass distribution in Testing Set:")
print(test_class_percentage)

Data split 45,55

In [2]:
# Split features and target variable - Train = 45 % Test = 55%
X = df_encoded.drop(['Is Fraudulent','Date'], axis=1)
y = df_encoded['Is Fraudulent']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

#Apply standard scalar
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)


print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

In [3]:
print(f"Training dataset: X_train: {X_train.shape}, y_train: {y_train.shape}\n{'_'*58}")
print(f"Testing data set: X_test: {X_test.shape}, y_test: {y_test.shape}")

NameError: name 'X_train' is not defined

In [ ]:
# Get distributions - Calculate the percentage of each class in the training set
train_class_percentage = y_train.value_counts(normalize=True) * 100
print("Class distribution in Training Set:")
print(train_class_percentage)

test_class_percentage = y_test.value_counts(normalize=True) * 100
print("\nClass distribution in Testing Set:")
print(test_class_percentage)

Data split 60,40

In [ ]:
# Split features and target variable - Train = 60 % Test = 40 %
X = df_encoded.drop(['Is Fraudulent','Date'], axis=1)
y = df_encoded['Is Fraudulent']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

#Apply standard scalar
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)


print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

In [ ]:
print(f"Training dataset: X_train: {X_train.shape}, y_train: {y_train.shape}\n{'_'*58}")
print(f"Testing data set: X_test: {X_test.shape}, y_test: {y_test.shape}")

NameError: name 'X_train' is not defined

In [ ]:
# Get distributions - Calculate the percentage of each class in the training set
train_class_percentage = y_train.value_counts(normalize=True) * 100
print("Class distribution in Training Set:")
print(train_class_percentage)

test_class_percentage = y_test.value_counts(normalize=True) * 100
print("\nClass distribution in Testing Set:")
print(test_class_percentage)

As you we can see, this data set is highly imbalanced in favour of non fraudulent transactions. One way of dealing with this is by oversampling using SMOTE.

SMOTE (Synthetic Minority Over-sampling Technique) creates synthetic points close to other points, upsampling the minority class in order to reach an equal balance between the minority and majority class.

In [ ]:
smote = SMOTE()
X_resampled_smote, y_resampled_smote = smote.fit_resample(X,y)
#Class distribution
print("Class distribution in the SMOTE oversampled training set:")
print(pd.Series(y_resampled_smote).value_counts(normalize=True) * 100)

As well as oversampling, undersampling also addresses the issue of class imbalance by reducing the representation of the majority (non fraud) class in the data set.

We will randomly remove classes from the majority class until there is balance.

Following this, we will then be able to train the machine learning models on both of the data sets.

In [ ]:
undersampler = RandomUnderSampler()
X_resampled_under, y_resampled_under = undersampler.fit_resample(X, y)
#Class distribution
print("Class distribution in the Random undersampled training set:")
print(pd.Series(y_resampled_under).value_counts(normalize=True) * 100)

In [ ]:
class_distr_before = pd.Series(y_train).value_counts(normalize=True) * 100
smote_class_distr = pd.Series(y_resampled_smote).value_counts(normalize=True) * 100
under_class_distr = pd.Series(y_resampled_under).value_counts(normalize=True) * 100

plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
sns.barplot(x=class_distr_before.index, y=class_distr_before.values)
plt.title('Imbalanced class distribution')
plt.xlabel('Is Fraudulent')
plt.ylabel('Percentage')

plt.subplot(1, 3, 2)
sns.barplot(x=smote_class_distr.index, y=smote_class_distr.values)
plt.title('SMOTE class distribution')
plt.xlabel('Is Fraudulent')
plt.ylabel('Percentage')

plt.subplot(1, 3, 3)
sns.barplot(x=under_class_distr.index, y=under_class_distr.values)
plt.title('Random undersampling class distribution')
plt.xlabel('Is Fraudulent')
plt.ylabel('Percentage')


plt.tight_layout()
plt.show()

Now that we have dealt with the imbalanced data we are able to build our model

As this is a classification problem we will be building a logistic regression model, and also a decision tree model

We will compare the scores of each model across the undersampled and oversampled data to see which one provides us with the best accuracy

# Logisitic Regression Model

In [ ]:
#Fit Both undersampled and oversampled data into logistic regression model and train it
LogReg_smote = LogisticRegression(max_iter=7000) # Adjust max_iter as needed
LogReg_under= LogisticRegression(max_iter=7000) # Adjust max_iter as needed

LogReg_smote.fit(X_resampled_smote, y_resampled_smote)
LogReg_under.fit(X_resampled_under, y_resampled_under)

#Test and predict
y_pred_smote = LogReg_smote.predict(X_resampled_smote)
y_pred_under = LogReg_under.predict(X_resampled_under)

roc_auc_smote = roc_auc_score(y_resampled_smote, y_pred_smote)
roc_auc_under = roc_auc_score(y_resampled_under, y_pred_under)


report_smote = classification_report(y_resampled_smote, y_pred_smote)
print("Classification Report for Logistic Regression Model (SMOTE):")
print(report_smote)
print("ROC AUC:", roc_auc_smote)

report_under = classification_report(y_resampled_under, y_pred_under)
print("\nClassification Report for Logistic Regression Model (Undersampling):")
print(report_under)
print("ROC AUC:", roc_auc_under)

In [ ]:
#Confusion matrix for both models
confusion_matrix_smote = confusion_matrix(y_resampled_smote, y_pred_smote)
confusion_matrix_under = confusion_matrix(y_resampled_under, y_pred_under)
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

#smote
sns.heatmap(confusion_matrix_smote, annot=True, fmt='d', cmap=plt.cm.Oranges,
            xticklabels=['Non-Fraud', 'Fraud'], yticklabels=['Non-Fraud', 'Fraud'], ax=axes[0])
axes[0].set_title('Confusion Matrix (SMOTE Logistic Regression)')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')

#undersampled
sns.heatmap(confusion_matrix_under, annot=True, fmt='d', cmap=plt.cm.Blues,
            xticklabels=['Non-Fraud', 'Fraud'], yticklabels=['Non-Fraud', 'Fraud'], ax=axes[1])
axes[1].set_title('Confusion Matrix (Undersampled Logistic Regression)')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')

plt.tight_layout()
plt.show()

<p>Four types of outcomes are possible while evaluating a classification model performance. These four outcomes are described below:-

True Positives (TP) – True Positives occur when we predict an observation belongs to a certain class and the observation actually belongs to that class.

True Negatives (TN) – True Negatives occur when we predict an observation does not belong to a certain class and the observation actually does not belong to that class.

False Positives (FP) – False Positives occur when we predict an observation belongs to a certain class but the observation actually does not belong to that class. This type of error is called Type I error.

False Negatives (FN) – False Negatives occur when we predict an observation does not belong to a certain class but the observation actually belongs to that class. This is a very serious error and it is called Type II error.

In [ ]:
# Confusion Matrix
cm_dt_smote = confusion_matrix(y_resampled_smote, y_pred_smote)

conf_matrix_dt_smote = pd.DataFrame(data=cm_dt_smote, columns=['Non-Fraud:0', 'Fraud:1'], index=['Non-Fraud:0', 'Fraud:1'])
plt.figure(figsize=(10,8))
sns.heatmap(conf_matrix_dt_smote, annot=True, fmt='d',cmap="YlGnBu")
plt.title('Classification for Churn')
plt.show()

In [ ]:
#Calculating measures for Logistic Regression based on our confusion matrix

TP = conf_matrix_dt_smote.iloc[0, 0]
TN = conf_matrix_dt_smote.iloc[1, 1]
FN = conf_matrix_dt_smote.iloc[0, 1]
FP = conf_matrix_dt_smote.iloc[1, 0]
total = TP + TN + FN + FP

print('Misclassification Rate: {:.1f}%'.format(100 * (FN + FP) / total))
print('Recall: {:.1f}%'.format(100 * TP / (TP + FN)))
print('Specificity: {:.1f}%'.format(100 * TN / (TN + FP)))
print('Precision: {:.1f}%'.format(100 * TP / (FP + TP)))

In [ ]:
# Confusion Matrix
cm_dt_under = confusion_matrix(y_resampled_under, y_pred_under)

conf_matrix_dt_under = pd.DataFrame(data=cm_dt_under, columns=['Non-Fraud:0', 'Fraud:1'], index=['Non-Fraud:0', 'Fraud:1'])
plt.figure(figsize=(10,8))
sns.heatmap(conf_matrix_dt_under, annot=True, fmt='d',cmap="YlGnBu")
plt.title('Classification for Churn')
plt.show()

In [ ]:
#Calculating measures for Logistic Regression based on our confusion matrix

TP = conf_matrix_dt_under.iloc[0, 0]
TN = conf_matrix_dt_under.iloc[1, 1]
FN = conf_matrix_dt_under.iloc[0, 1]
FP = conf_matrix_dt_under.iloc[1, 0]
total = TP + TN + FN + FP

print('Misclassification Rate: {:.1f}%'.format(100 * (FN + FP) / total))
print('Recall: {:.1f}%'.format(100 * TP / (TP + FN)))
print('Specificity: {:.1f}%'.format(100 * TN / (TN + FP)))
print('Precision: {:.1f}%'.format(100 * TP / (FP + TP)))

In [ ]:
# Normalize the confusion matrices
normalized_confusion_matrix_smote = confusion_matrix_smote / confusion_matrix_smote.sum(axis=1, keepdims=True)
normalized_confusion_matrix_under = confusion_matrix_under / confusion_matrix_under.sum(axis=1, keepdims=True)


fig, axes = plt.subplots(1, 2, figsize=(15, 6))


sns.heatmap(normalized_confusion_matrix_smote, annot=True, fmt='.2%', cmap=plt.cm.Oranges,
            xticklabels=['Non-Fraud', 'Fraud'], yticklabels=['Non-Fraud', 'Fraud'], ax=axes[0], cbar=True)
axes[0].set_title('Normalized Confusion Matrix (SMOTE Logistic Regression)')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')


sns.heatmap(normalized_confusion_matrix_under, annot=True, fmt='.2%', cmap=plt.cm.Blues,
            xticklabels=['Non-Fraud', 'Fraud'], yticklabels=['Non-Fraud', 'Fraud'], ax=axes[1], cbar=True)
axes[1].set_title('Normalized Confusion Matrix (Undersampled Logistic Regression)')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')

plt.tight_layout()
plt.show()

# Fine-Tuning Hyperparameters

**Decision Tree**

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder


#Preprocessing: Encode categorical features
label_encoders = {}
for feature in data.columns:
    if data[feature].dtype == 'object':
        label_encoders[feature] = LabelEncoder()
        data[feature] = label_encoders[feature].fit_transform(data[feature].astype(str))

# Convert Timestamp data to numerical features
#data['Date'] = data['Date'].astype('datetime64[ns]').astype(np.int64) // 10**9  # Convert Timestamp to Unix timestamp (seconds)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#X_train.drop("Date", axis=1, inplace=True)
#X_test.drop("Date", axis=1, inplace=True)

# Initialize the Decision Tree classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Define the hyperparameters grid
param_grid = {
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],   # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]      # Minimum number of samples required to be at a leaf node
}

# Initialize GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(estimator=dt_classifier, param_grid=param_grid, cv=5, scoring='accuracy')

# Perform hyperparameter tuning
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Initialize Decision Tree classifier with best hyperparameters
best_dt_classifier = DecisionTreeClassifier(**best_params, random_state=42)

# Train the model with the best hyperparameters
best_dt_classifier.fit(X_train, y_train)

# Evaluate the model
accuracy = best_dt_classifier.score(X_test, y_test)
print("Accuracy:", accuracy)


**5.Performance Evaluation**

In [ ]:


# Split features and target variable
X = df_encoded.drop('Is Fraudulent', axis=1)
y = df_encoded['Is Fraudulent']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#X_train["Date"] = X_train["Date"].astype(float)
#X_test["Date"] = X_test["Date"].astype(float)

X_train.drop("Date", axis=1, inplace=True)
X_test.drop("Date", axis=1, inplace=True)

# Train Logistic Regression model
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

# Train Decision Tree model
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)

# Make predictions
lr_preds = lr_model.predict(X_test)
dt_preds = dt_model.predict(X_test)

# Evaluate Logistic Regression model
lr_accuracy = accuracy_score(y_test, lr_preds)
lr_precision = precision_score(y_test, lr_preds, average='weighted',zero_division=1)
lr_recall = recall_score(y_test, lr_preds, average='weighted')
lr_f1 = f1_score(y_test, lr_preds, average='weighted')
lr_roc_auc = roc_auc_score(y_test, lr_preds, average='weighted')

# Evaluate Decision Tree model
dt_accuracy = accuracy_score(y_test, dt_preds)
dt_precision = precision_score(y_test, dt_preds, average='weighted',zero_division=1)
dt_recall = recall_score(y_test, dt_preds, average='weighted')
dt_f1 = f1_score(y_test, dt_preds, average='weighted')
dt_roc_auc = roc_auc_score(y_test, dt_preds, average='weighted')

# Print the evaluation metrics
print("Logistic Regression Metrics:")
print("Accuracy:", lr_accuracy)
print("Precision:", lr_precision)
print("Recall:", lr_recall)
print("F1-score:", lr_f1)
print("ROC-AUC score:", lr_roc_auc)

print("\nDecision Tree Metrics:")
print("Accuracy:", dt_accuracy)
print("Precision:", dt_precision)
print("Recall:", dt_recall)
print("F1-score:", dt_f1)
print("ROC-AUC score:", dt_roc_auc)

# Assignment Part 2

## 1. Model Building
## Build ML models using K-Nearest Neighbors (K-NN), Support Vector Machine (SVM), Naive Bayesian, Random Forest, and Adaboost

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

# Convert 'Date' column to datetime
#df_encoded["Date"] = pd.to_datetime(df_encoded["Date"])

# Perform one-hot encoding on categorical variables
#data = pd.get_dummies(data, columns=['Card Type', 'MCC Category', 'Location', 'Device', 'Merchant Reputation', 'Online Transactions Frequency'])



# Split features and target variable
X = df_encoded.drop('Is Fraudulent', axis=1)
y = df_encoded['Is Fraudulent']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#X_train["Date"] = X_train["Date"].astype(float)
#X_test["Date"] = X_test["Date"].astype(float)

X_train.drop("Date", axis=1, inplace=True)
X_test.drop("Date", axis=1, inplace=True)

# Scale numerical features
#scaler = StandardScaler()
#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.transform(X_test)

# K-Nearest Neighbors (K-NN)
knn = KNeighborsClassifier(n_neighbors=5)
# Training the algorithm and prediction on testing data & Evaluation
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_pred)
print("KNN Accuracy:", knn_accuracy)
lr_precision = precision_score(y_test, knn_pred,zero_division=1)
print("Precision Score:", lr_precision)
lr_recall = recall_score(y_test, knn_pred)
print("Precision Score:", lr_recall)
lr_f1 = f1_score(y_test, knn_pred)
print("Precision Score:", lr_f1)


#Support Vector Machine
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_pred)
print("SVM Accuracy:", svm_accuracy)
svc_precision = precision_score(y_test, svm_pred,zero_division=1)
print("Precision Score:", svc_precision)
svc_recall = recall_score(y_test, svm_pred)
print("Precision Score:", svc_recall)
svc_f1 = f1_score(y_test, svm_pred)
print("Precision Score:", svc_f1)


# Naive Bayesian
nb = GaussianNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_test)
nb_accuracy = accuracy_score(y_test, nb_pred)
print("Naive Bayesian Accuracy:", nb_accuracy)
nb_precision = precision_score(y_test, nb_pred,zero_division=1)
print("Precision Score:", nb_precision)
nb_recall = recall_score(y_test, nb_pred)
print("Precision Score:", nb_recall)
nb_f1 = f1_score(y_test, nb_pred)
print("Precision Score:", nb_f1)


# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_pred)
print("Random Forest Accuracy:", rf_accuracy)
rf_precision = precision_score(y_test, rf_pred,zero_division=1)
print("Precision Score:", rf_precision)
rf_recall = recall_score(y_test, rf_pred)
print("Precision Score:", rf_recall)
rf_f1 = f1_score(y_test, rf_pred)
print("Precision Score:", rf_f1)



# Adaboost
ada = AdaBoostClassifier(n_estimators=100, random_state=42)
ada.fit(X_train, y_train)
ada_pred = ada.predict(X_test)
ada_accuracy = accuracy_score(y_test, ada_pred)
print("Adaboost Accuracy:", ada_accuracy)
ada_precision = precision_score(y_test, ada_pred,zero_division=1)
print("Precision Score:", ada_precision)
ada_recall = recall_score(y_test, ada_pred)
print("Precision Score:", ada_recall)
ada_f1 = f1_score(y_test, ada_pred)
print("Precision Score:", ada_f1)


# For regression
k = 5  # number of neighbors to consider
knn_regressor = KNeighborsRegressor(n_neighbors=k)



# For regression
knn_regressor.fit(X_train, y_train)

# For regression
y_pred_reg = knn_regressor.predict(X_test)


# For regression
mse = mean_squared_error(y_test, y_pred_reg)
print("Mean Squared Error:", mse)
"""
# Example of hyperparameter tuning for classification
for k in range(1, 11):
    knn_classifier = KNeighborsClassifier(n_neighbors=k)
    knn_classifier.fit(X_train, y_train)
    y_pred_class = knn_classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_class)
    print("K =", k, "Accuracy:", accuracy)




# Initialize the logistic regression model
#model = LogisticRegression()

# Train the model
#model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_train)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

print("\nClassification Report:")
print(classification_report(y_test,zero_division=1))
"""

In [ ]:
# K-Nearest Neighbors (K-NN)
knn = KNeighborsClassifier(n_neighbors=5)
# Training the algorithm and prediction on testing data & Evaluation
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_pred)
print("KNN Accuracy:", knn_accuracy)
lr_precision = precision_score(y_test, knn_pred,zero_division=1)
print("Precision Score:", lr_precision)
lr_recall = recall_score(y_test, knn_pred)
print("Recall:", lr_recall)
lr_f1 = f1_score(y_test, knn_pred)
print("F1 Score:", lr_f1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, knn_pred,zero_division=1))

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

LABELS=['Non-Fraud', 'Fraud']
# Create confusion matrix
confusion_matrix = confusion_matrix(y_test, knn_pred)

# Create subplots
fig, ax = plt.subplots(1,  figsize=(8, 6))

sns.heatmap(confusion_matrix, annot=True, fmt='d', cmap='viridis',
            xticklabels=LABELS, yticklabels=LABELS)
ax.set_title('Confusion Matrix')
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')



# Adjust layout and display
plt.tight_layout()
plt.show()

In [ ]:
# 3) Fine-Tuning Hyperparameters
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for K-NN
param_grid_knn = {'n_neighbors': [3, 5, 7, 9, 11]}

# Initialize K-NN classifier
knn = KNeighborsClassifier()

# Initialize GridSearchCV with K-NN classifier and parameter grid
grid_search_knn = GridSearchCV(knn, param_grid_knn, cv=5, scoring='accuracy')

# Perform GridSearchCV to find the best parameters
grid_search_knn.fit(X_train, y_train)

# Get the best parameters and the best accuracy score
best_params_knn = grid_search_knn.best_params_
best_score_knn = grid_search_knn.best_score_

print("Best Parameters for K-NN:", best_params_knn)
print("Best Accuracy Score for K-NN:", best_score_knn)


In [ ]:
#Support Vector Machine
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_pred)
print("SVM Accuracy:", svm_accuracy)
svc_precision = precision_score(y_test, svm_pred,zero_division=1)
print("Precision Score:", svc_precision)
svc_recall = recall_score(y_test, svm_pred)
print("Recall:", svc_recall)
svc_f1 = f1_score(y_test, svm_pred)
print("F1 Score:", svc_f1)



In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, svm_pred,zero_division=1))

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

LABELS=['Non-Fraud', 'Fraud']
# Create confusion matrix
confusion_matrix = confusion_matrix(y_test, svm_pred)

# Create subplots
fig, ax = plt.subplots(1,  figsize=(8, 6))

sns.heatmap(confusion_matrix, annot=True, fmt='d', cmap='viridis',
            xticklabels=LABELS, yticklabels=LABELS)
ax.set_title('Confusion Matrix')
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')



# Adjust layout and display
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

fpr, tpr, threshold = roc_curve(y_test, svm_pred)


In [ ]:


def plot_roc_curve(fpr, tpr, label=None):
    plt.figure(figsize=(8, 6))
    plt.title('ROC Curve', fontsize=15)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.xticks(np.arange(0, 1, 0.05), rotation=90)
    plt.xlabel('False Positive Rates', fontsize=15)
    plt.ylabel('True Positive Rates', fontsize=15)
    plt.legend(loc='best')

    plt.show()

In [ ]:

# First, getting the auc score
svc_auc = roc_auc_score(y_test, svm_pred)

# Now, let's get the fpr and tpr
fpr, tpr, threshold = roc_curve(y_test, svm_pred)

# Now, let's draw the curve
plot_roc_curve(fpr, tpr, 'AUC: %.3f' % svc_auc)

In [ ]:
#Tune Hyperparameters

# For Kernel = rbf
tuned_rbf = {'kernel': ['rbf'], 'gamma': [
    1e-2, 1e-3, 1e-4, 1e-5], 'C': [0.001, 0.10, 0.1, 10, 25, 50, 100, 1000]}

# For kernel = sigmoid
tuned_sigmoid = {'kernel': ['sigmoid'], 'gamma': [
    1e-2, 1e-3, 1e-4, 1e-5], 'C': [0.001, 0.10, 0.1, 10, 25, 50, 100, 1000]}

# For kernel = linear
tuned_linear = {'kernel': ['linear'], 'C': [
    0.001, 0.10, 0.1, 10, 25, 50, 100, 1000]}


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

rs_rbf = RandomizedSearchCV(estimator=SVC(probability=True),
        param_distributions=tuned_rbf, n_iter=500, n_jobs=4, scoring='roc_auc')

rs_sigmoid = RandomizedSearchCV(estimator=SVC(probability=True),
        param_distributions=tuned_sigmoid, n_iter=500, n_jobs=4, scoring='roc_auc')

rs_linear = RandomizedSearchCV(estimator=SVC(probability=True),
        param_distributions=tuned_linear, n_iter=500, n_jobs=4, scoring='roc_auc')

**For kernel rbf:**

In [ ]:
rs_rbf.fit(X_train, y_test)

In [ ]:
# Naive Bayesian
nb = GaussianNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_test)
nb_accuracy = accuracy_score(y_test, nb_pred)
print("Naive Bayesian Accuracy:", nb_accuracy)
nb_precision = precision_score(y_test, nb_pred,zero_division=1)
print("Precision Score:", nb_precision)
nb_recall = recall_score(y_test, nb_pred)
print("Recall:", nb_recall)
nb_f1 = f1_score(y_test, nb_pred)
print("F1:", nb_f1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, nb_pred,zero_division=1))

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

LABELS=['Non-Fraud', 'Fraud']
# Create confusion matrix
confusion_matrix = confusion_matrix(y_test, nb_pred)

# Create subplots
fig, ax = plt.subplots(1,  figsize=(8, 6))

sns.heatmap(confusion_matrix, annot=True, fmt='d', cmap='viridis',
            xticklabels=LABELS, yticklabels=LABELS)
ax.set_title('Confusion Matrix')
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')



# Adjust layout and display
plt.tight_layout()
plt.show()

In [ ]:
# 3) Fine-Tuning Hyperparameters
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define the parameter grid for Naive Bayes
param_grid_nb = {
    'var_smoothing': [1e-9, 1e-8, 1e-7]  # Adjust var_smoothing parameter
}

# Initialize Naive Bayes classifier
nb = GaussianNB()

# Initialize GridSearchCV with Naive Bayes classifier and parameter grid
grid_search_nb = GridSearchCV(nb, param_grid_nb, cv=5, scoring='accuracy')

# Perform GridSearchCV to find the best parameters
grid_search_nb.fit(X_train, y_train)

# Get the best parameters and the best accuracy score
best_params_nb = grid_search_nb.best_params_
best_score_nb = grid_search_nb.best_score_

print("Best Parameters for Naive Bayes:", best_params_nb)
print("Best Accuracy Score for Naive Bayes:", best_score_nb)

# Use the best parameters to train the final Naive Bayes model
nb_best = GaussianNB(var_smoothing=best_params_nb['var_smoothing'])
nb_best.fit(X_train, y_train)

# Predict on the test set using the final model
nb_pred = nb_best.predict(X_test)

# Evaluate the final model
nb_accuracy = accuracy_score(y_test, nb_pred)
print("Naive Bayesian Accuracy:", nb_accuracy)
nb_precision = precision_score(y_test, nb_pred, zero_division=1)
print("Precision Score:", nb_precision)
nb_recall = recall_score(y_test, nb_pred)
print("Recall:", nb_recall)
nb_f1 = f1_score(y_test, nb_pred)
print("F1:", nb_f1)


In [ ]:


# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_pred)
print("Random Forest Accuracy:", rf_accuracy)
rf_precision = precision_score(y_test, rf_pred,zero_division=1)
print("Precision Score:", rf_precision)
rf_recall = recall_score(y_test, rf_pred)
print("Recall:", rf_recall)
rf_f1 = f1_score(y_test, rf_pred)
print("F1:", rf_f1)


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, rf_pred,zero_division=1))

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

LABELS=['Non-Fraud', 'Fraud']
# Create confusion matrix
confusion_matrix = confusion_matrix(y_test, rf_pred)

# Create subplots
fig, ax = plt.subplots(1,  figsize=(8, 6))

sns.heatmap(confusion_matrix, annot=True, fmt='d', cmap='viridis',
            xticklabels=LABELS, yticklabels=LABELS)
ax.set_title('Confusion Matrix')
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')



# Adjust layout and display
plt.tight_layout()
plt.show()

In [ ]:
# 3) Fine-Tuning Hyperparameters
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define the parameter grid for Random Forest
param_grid_rf = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],   # Maximum depth of the trees
    'min_samples_split': [2, 5, 10],   # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]      # Minimum number of samples required to be at a leaf node
}

# Initialize Random Forest classifier
rf = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV with Random Forest classifier and parameter grid
grid_search_rf = GridSearchCV(rf, param_grid_rf, cv=5, scoring='accuracy')

# Perform GridSearchCV to find the best parameters
grid_search_rf.fit(X_train, y_train)

# Get the best parameters and the best accuracy score
best_params_rf = grid_search_rf.best_params_
best_score_rf = grid_search_rf.best_score_

print("Best Parameters for Random Forest:", best_params_rf)
print("Best Accuracy Score for Random Forest:", best_score_rf)

# Use the best parameters to train the final Random Forest model
rf_best = RandomForestClassifier(random_state=42, **best_params_rf)
rf_best.fit(X_train, y_train)

# Predict on the test set using the final model
rf_pred = rf_best.predict(X_test)

# Evaluate the final model
rf_accuracy = accuracy_score(y_test, rf_pred)
print("Random Forest Accuracy:", rf_accuracy)
rf_precision = precision_score(y_test, rf_pred, zero_division=1)
print("Precision Score:", rf_precision)
rf_recall = recall_score(y_test, rf_pred)
print("Recall:", rf_recall)
rf_f1 = f1_score(y_test, rf_pred)
print("F1:", rf_f1)


In [ ]:


# Adaboost
ada = AdaBoostClassifier(n_estimators=100, random_state=42)
ada.fit(X_train, y_train)
ada_pred = ada.predict(X_test)
ada_accuracy = accuracy_score(y_test, ada_pred)
print("Adaboost Accuracy:", ada_accuracy)
ada_precision = precision_score(y_test, ada_pred,zero_division=1)
print("Precision Score:", ada_precision)
ada_recall = recall_score(y_test, ada_pred)
print("Recall:", ada_recall)
ada_f1 = f1_score(y_test, ada_pred)
print("F1:", ada_f1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, ada_pred,zero_division=1))

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss
#Model evaluation based on K-fold cross-validation using cross_val_score() function
# 10-fold cross-validation logistic regression
ada_fold = AdaBoostClassifier()
# Use cross_val_score function
# We are passing the entirety of X and y, not X_train or y_train, it takes care of splitting the data
# cv=10 for 10 folds
# scoring = {'accuracy', 'neg_log_loss', 'roc_auc'} for evaluation metric - althought they are many
scores_accuracy = cross_val_score(ada_fold, X_train, y_train, cv=10, scoring='accuracy')
scores_log_loss = cross_val_score(ada_fold, X_train, y_train, cv=10, scoring='neg_log_loss')
scores_auc = cross_val_score(ada_fold, X_train, y_train, cv=10, scoring='roc_auc')
print('K-fold cross-validation results:')
print(ada_fold.__class__.__name__+" average accuracy is %2.3f" % scores_accuracy.mean())
print(ada_fold.__class__.__name__+" average log_loss is %2.3f" % -scores_log_loss.mean())
print(ada_fold.__class__.__name__+" average auc is %2.3f" % scores_auc.mean())

In [ ]:
#Model evaluation based on K-fold cross-validation using cross_validate() function
from sklearn.model_selection import cross_validate

scoring = {'accuracy': 'accuracy', 'log_loss': 'neg_log_loss', 'auc': 'roc_auc'}

modelCV = AdaBoostClassifier()

results = cross_validate(modelCV, X_train, y_train, cv=10, scoring=list(scoring.values()),
                         return_train_score=False)

print('K-fold cross-validation results:')
for sc in range(len(scoring)):
    print(modelCV.__class__.__name__+" average %s: %.3f (+/-%.3f)" % (list(scoring.keys())[sc], -results['test_%s' % list(scoring.values())[sc]].mean()
                               if list(scoring.values())[sc]=='neg_log_loss'
                               else results['test_%s' % list(scoring.values())[sc]].mean(),
                               results['test_%s' % list(scoring.values())[sc]].std()))

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

LABELS=['Non-Fraud', 'Fraud']
# Create confusion matrix
confusion_matrix = confusion_matrix(y_test, ada_pred)

# Create subplots
fig, ax = plt.subplots(1,  figsize=(8, 6))

sns.heatmap(confusion_matrix, annot=True, fmt='d', cmap='viridis',
            xticklabels=LABELS, yticklabels=LABELS)
ax.set_title('Confusion Matrix')
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')



# Adjust layout and display
plt.tight_layout()
plt.show()

In [ ]:
# 3) Fine-Tuning Hyperparameters
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define the parameter grid for AdaBoostClassifier
param_grid_ada = {
    'n_estimators': [50, 100, 200],           # Number of weak learners
    'learning_rate': [0.01, 0.1, 1.0]         # Learning rate
}

# Initialize AdaBoostClassifier
ada = AdaBoostClassifier(random_state=42)

# Initialize GridSearchCV with AdaBoostClassifier and parameter grid
grid_search_ada = GridSearchCV(ada, param_grid_ada, cv=5, scoring='accuracy')

# Perform GridSearchCV to find the best parameters
grid_search_ada.fit(X_train, y_train)

# Get the best parameters and the best accuracy score
best_params_ada = grid_search_ada.best_params_
best_score_ada = grid_search_ada.best_score_

print("Best Parameters for AdaBoost:", best_params_ada)
print("Best Accuracy Score for AdaBoost:", best_score_ada)

# Use the best parameters to train the final AdaBoost model
ada_best = AdaBoostClassifier(random_state=42, **best_params_ada)
ada_best.fit(X_train, y_train)

# Predict on the test set using the final model
ada_pred = ada_best.predict(X_test)

# Evaluate the final model
ada_accuracy = accuracy_score(y_test, ada_pred)
print("AdaBoost Accuracy:", ada_accuracy)
ada_precision = precision_score(y_test, ada_pred, zero_division=1)
print("Precision Score:", ada_precision)
ada_recall = recall_score(y_test, ada_pred)
print("Recall:", ada_recall)
ada_f1 = f1_score(y_test, ada_pred)
print("F1:", ada_f1)
